# Structure Optimization

### Import modules

In [1]:
import torch
import os
import sys
sys.path.append(os.path.abspath(".."))

import numpy as np
from ase.build import bulk
from ase.units import GPa
from ase.calculators.emt import EMT
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running MatterSim on {device}")

Running MatterSim on cpu


### Available Models

|                    | mattersim-v1.0.0-1M   | mattersim-v1.0.0-5M     |
| ------------------ | --------------------- | ----------------------- |
| Training Data Size | 3M                    | 6M                      |
| Model Parameters   | 880K                  | 4.5M                    |

In [3]:
# model = 1 or 5 only
model = 5

### Si structure

##### Setup Si structure to relax

In [4]:
# initialize the structure of silicon
si = bulk("Si", "diamond", a=5.43)

print(f"si positions: {si.positions}")

# attach the calculator to the atoms object
si.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)
print(f"\nMatterSimCalculator loaded with {model}M model\n")

si positions: [[0.     0.     0.    ]
 [1.3575 1.3575 1.3575]]

MatterSimCalculator loaded with 5M model



2025-02-06 02:00:20.009 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-06 02:02:39.038 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-06 02:02:39.138 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-06 02:02:39.212 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-06 02:02:39.284 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-06 02:02:39.354 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-06 02:02:39.422 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pt

/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [5]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.814008712768555
Energy per atom (eV/atom)   = -5.407004356384277
Forces of first atom (eV/A) = [-1.5469268e-06 -1.4500692e-06 -1.7108396e-06]
Stress[0][0] (x-x) (eV/A^3) = -0.011929342470998954
Stress[0][0] (x-x) (GPa)    = -1.9112913608551025


In [6]:
# initialize the relaxation object
relaxer = Relaxer(
    optimizer="BFGS", # the optimization method
    filter="ExpCellFilter", # filter to apply to the cell
    constrain_symmetry=True, # whether to constrain the symmetry
)

relaxed_structure = relaxer.relax(si, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 02:00:28      -10.814009        0.477481
BFGS:    1 02:00:28      -10.818985        0.015249
BFGS:    2 02:00:28      -10.818995        0.000545


In [7]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (eV/A^3)       = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (GPa)          = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.818995475769043
Energy per atom (eV/atom)   = -5.4094977378845215
Forces of first atom (eV/A) = [-6.0535967e-09 -9.4482675e-07 -7.5111166e-07]
Stress[0][0] (eV/A^3)       = -1.3320548554476158e-05
Stress[0][0] (GPa)          = -0.0021341871470212936


##### Predict 1000 Si structures

In [8]:
# initialize the structure of silicon
bulk_predictions =[]
for _ in range(1000):
    si = bulk("Si", "diamond", a=5.43)
    si.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)
    bulk_predictions.append(si.get_potential_energy())

/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [ ]:
for i in range(1000):  # Check 1000 predictions
    assert bulk_predictions[i] == bulk_predictions[0], f"Energy of structure {i} is different {bulk_predictions[i]}!"

##### Perturb Si structure & perform relaxation

In [10]:
# initialize the structure of silicon
si = bulk("Si", "diamond", a=5.43)

# perturb the structure
si.positions += 0.1 * np.random.randn(len(si), 3)

# attach the calculator to the atoms object
si.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [11]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (eV/A^3)       = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (GPa)          = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.747528076171875
Energy per atom (eV/atom)   = -5.3737640380859375
Forces of first atom (eV/A) = [-0.73042315  0.12353308  1.0355029 ]
Stress[0][0] (eV/A^3)       = -0.014679966952319724
Stress[0][0] (GPa)          = -2.351989984512329


In [12]:
relaxed_structure = relaxer.relax(si, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 02:05:15      -10.747528        1.273203
BFGS:    1 02:05:15      -10.800085        0.546508
BFGS:    2 02:05:15      -10.809313        0.253247
BFGS:    3 02:05:15      -10.814323        0.294246
BFGS:    4 02:05:15      -10.817862        0.191169
BFGS:    5 02:05:15      -10.818904        0.044287
BFGS:    6 02:05:15      -10.818983        0.009406


In [13]:
print(f"Energy (eV)                 = {si.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {si.get_potential_energy()/len(si)}")
print(f"Forces of first atom (eV/A) = {si.get_forces()[0]}")
print(f"Stress[0][0] (eV/A^3)       = {si.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (GPa)          = {si.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.81898307800293
Energy per atom (eV/atom)   = -5.409491539001465
Forces of first atom (eV/A) = [ 0.00288048 -0.00715344 -0.00521126]
Stress[0][0] (eV/A^3)       = -3.408629618114821e-05
Stress[0][0] (GPa)          = -0.005461226683109998


In [14]:
relaxed_structure[1].cell

Cell([[0.0017883150027152967, 2.734353991893051, 2.7336770503427674], [2.7356230108771293, 0.00018259608879849605, 2.7352827692566755], [2.7343563941110824, 2.7346930940409258, 0.001449212854838607]])

### C structure

In [15]:
# generate a random 'a' to start with.
a = np.random.uniform(1,10)
a

8.467628022506664

In [16]:
c = bulk('C', 'diamond', a=a)
c.positions

array([[0.        , 0.        , 0.        ],
       [2.11690701, 2.11690701, 2.11690701]])

In [17]:
c.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

In [18]:
print(f"Energy (eV)                 = {c.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {c.get_potential_energy()/len(c)}")
print(f"Forces of first atom (eV/A) = {c.get_forces()[0]}")
print(f"Stress[0][0] (GPa)          = {c.get_stress(voigt=False)[0][0] / GPa}")
print(f"Stress[0][0] (eV/A^3)       = {c.get_stress(voigt=False)[0][0]}")

Energy (eV)                 = -3.3781771659851074
Energy per atom (eV/atom)   = -1.6890885829925537
Forces of first atom (eV/A) = [ 1.5157275e-07 -1.5157275e-07 -6.1001629e-08]
Stress[0][0] (GPa)          = -0.06572873890399933
Stress[0][0] (eV/A^3)       = -0.00041024652370210974


In [19]:
relaxed_structure = relaxer.relax(c, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 02:05:15       -3.378177        0.062269
BFGS:    1 02:05:15       -3.378345        0.062790
BFGS:    2 02:05:15       -3.426280        0.385191
BFGS:    3 02:05:15       -4.159799        4.587741
BFGS:    4 02:05:15       -4.869390        0.000000


In [20]:
print(f"Energy (eV)                 = {c.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {c.get_potential_energy()/len(c)}")
print(f"Forces of first atom (eV/A) = {c.get_forces()[0]}")
print(f"Stress[0][0] (GPa)          = {c.get_stress(voigt=False)[0][0] / GPa}")
print(f"Stress[0][0] (eV/A^3)       = {c.get_stress(voigt=False)[0][0]}")

Energy (eV)                 = -4.869390487670898
Energy per atom (eV/atom)   = -2.434695243835449
Forces of first atom (eV/A) = [-0. -0. -0.]
Stress[0][0] (GPa)          = 0.0
Stress[0][0] (eV/A^3)       = 0.0


In [21]:
relaxed_structure[1].cell

Cell([[-2.1859403049360556e-16, 5.824592389348445, 5.824592389348445], [5.824592389348445, -2.8825007694067325e-16, 5.824592389348445], [5.824592389348445, 5.824592389348446, -5.50416761226281e-17]])